# DFT - Aplicación práctica - Tonos teléfono



Los teléfono analógicos empleaban un sistema de tonos para comunicarse con la central y enviar el número de teléfono marcado. En la actualidad, con los teléfonos digitales, este método está en desuso pero aun se emplea para los sistemas tipo contestador en los que es necesario escoger opciones (ej. "Presione 1 para hacer X").

El sistema empleado era el siguiente: los números y símbolos estaban colocados en forma de matriz y cada uno de esos números tenía asociado 2 frecuencias (alta y baja) que generaban los tonos  en el momento que se pulsaban. La tabla de frecuencias era la siguiente:


|            | **1209 Hz** | **1336 Hz** | **1477 Hz** |
|------------|:-----------:|:-----------:|:-----------:|
| **697 Hz** |      1      |      2      |      3      |
| **770 Hz** |      4      |      5      |      6      |
| **852 Hz** |      7      |      8      |      9      |
| **941 Hz** |      *      |      0      |      #      |


Las combinaciones de frecuencia en esa tabla se seleccionaron para ser "coprimas", es decir, ninguna frecuencia es múltiplo de ninguna otra. Esto reduce la probabilidad de errores en caso de interferencias. En el momento en que una tecla es presionada se generan y envían las señales con las frecuencias oportunas.
Ejemplo: si se presiona el digito 1 la señal generada es


$$
    \large x(t) = A\sin(2\pi\cdot 1209\cdot t) + A\sin(2\pi\cdot697\cdot t)
$$


Según especificaciones oficiales:
* Cada tono debería de durar mínimo 65ms
* 2 tonos consecutivos deberían de estar separados por un silencion de, al menos, 65ms.


En nuestro caso estamos trabajando con señales discretas y es en lo que vamos a centrar el Notebook. El objetivo es realizar un codificador/descodificador de tonos de teléfono empleando la DFT. 
Vamos a emplear una frecuencia de reloj de $F_s=24000Hz$. Simplificando, significa que nuestra tarjeta de sonido ejecutará $F_s$ muestras por segundo. Por lo que para crear los tonos y pasar del continuo al discreto haremos lo siguiente:

$\large x[n] = A\sin(2\pi\,\frac{F_{baja}}{F_s}\, n) + A\sin(2\pi\,\frac{F_{alta}}{F_s}\,n)$


Lo primero que vamos a hacer es crear el codificador

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython
plt.rcParams["figure.figsize"] = (14,4)

### Ejercicio

Completa la siguiente función para crear un codificador de numeros->tonos


In [2]:
#FS es la frecuencia de muestreo
FS = 24000

FRECUENCIAS = {
        '1': (697, 1209), '2': (697, 1336), '3': (697, 1477),
        '4': (770, 1209), '5': (770, 1336), '6': (770, 1477),
        '7': (852, 1209), '8': (852, 1336), '9': (852, 1477),
        '*': (941, 1209), '0': (941, 1336), '#': (941, 1477),        
    }


#constante para generar un numero de muestras en base a la frecuencia de muestreo 
#para las pulsaciones de teclado y los espacios vacíos (la aproximacion a 65ms)
ANCHO_TONOS=0.1 

def tonos(numeros,A=1):
    
   
    n = np.arange(0, int(ANCHO_TONOS * FS))#las muestras que vamos a generar
    
    rdo = np.array([])
    
    ##recorremos el array de numeros y lo transformamos en los tonos
    # Cada número es la suma de 2 señales y un espacio en blanco del mismo ancho con ceros,
    # es decir, la suma de las señales concatenada con un array de ceros
    #Completar la función
    ####
    ####
    ####
    
    return rdo
   

In [3]:
#escuchamos el resultado
x_tonos=tonos('981#123')
IPython.display.Audio(x_tonos, rate=FS)


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
plt.plot(x_tonos)

### Ejercicio: Añade ruido a la señal
Para ser más realista vamos a agregar ruido a la señal de tonos.

Podéis usar la función random.normal de numpy pero tened cuidado con la $\sigma$ que usais para no modificar completamente la señal original. Ese valor dependerá de la amplitud que tenéis para las sinusoidales

In [ ]:
#Ejercicio añadir ruido

In [ ]:
#visualizamos los tonos con ruido
plt.plot(x_tonos)

### Ejercicio
Desarrolla una función que separe del array de tonos los números de las espacios vacíos. La función tendría que recibir el array de tonos y devolver un array de tuplas. Cada tupla contiene el primer y último índice de cada número o símbolo que tenemos que decodificar

[(1, 2400), (4800,7200),...]

Tendremos que generar *ventanas* y calcular la **energía local** de esa ventana para ver si tiene una energía superior a un threshold o no (por culpa del posible ruído no podemos filtrar por el cero) y así separar los tonos de los espacios vacíos. Recordamos el cálculo de la energía para una señal finita:

$E_{local}=\sum\limits_{0}^{N-1}|x[n]|^2$

Lo vamos a simplificar mucho creando ventanas del ancho de los tonos (FS$\cdot$ancho_tono). Lo realista sería emplear ventanas más pequeñas para ir analizando la secuencia porque en la realidad puede tener interferencias y no estar todo *perfecto*.

Siguiendo con la simplificación, podéis hacer un reshape de la señal  con los tonos empleando el tamaño de la ventana. La idea es crear una matriz con un número de columnas igual al ancho de la ventana que estamos empleando, de esa forma se creará una matriz con tantas filas como ventanas estamos analizando.

In [ ]:
#Ejercicio separar tonos

### Ejercicio

Crea un decodificador de los tonos. La función recibirá la señal con los tonos y el array de tuplas con los índices inicial y final de cada tono. Empleando la DFT podéis encontrar las frecuencias de las que está compuesto  cada tono y en consecuencia obtener el número o símbolo que lo generó. Para pasar de la frecuencia discreta a la del *mundo real*, tenéis que multiplicar por $F_s$ (la operación contraria que se hizo para la generación de los tonos con las  sinusoidales):

$F_{baja}=F_s f_{ruido-baja}~ H_z$

$F_{alta}=F_s f_{ruido-alta}~ H_z$

Podemos usar la función que creamos en la práctica anterior para buscar las frecuencias de los 'n' picos máximos.

Una vez obtenida la frecuencia en el *mundo real*, tenemos que compararla con las frecuencias posibles. Fijaos que existe una lista de posibles frecuencias bajas y otro de posibles frecuencias altas. La combinación es la que da el número marcado. Al tener ruido en la señal vais a tener que buscar la frecuencia más cercana de las posibles a la que habéis encontrado.

In [ ]:
def get_frequencies(X, freq,elements):
        X_modules=abs(X.copy())
        #argsort devuelve una lista ordenada con los índices con los valores máximos
        #al negarlo consigo darle la vuelta al orden y me es más fácil recuperar lo que 
        #me interesa. 
        idx=  (-X_modules).argsort()
        #Creo un array que solo tiene las frecuencias positivas ordenadas y devuelvo los
        # "elements" que me piden
        return [ (X_modules[i], freq[i]) for i in idx if freq[i]>0][:elements]     

#Ejercicio obtener números